In [1]:
# https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py

"""
Optuna example that optimizes multi-layer perceptrons using PyTorch.

In this example, we optimize the validation accuracy of fashion product recognition using
PyTorch and FashionMNIST. We optimize the neural network architecture as well as the optimizer
configuration. As it is too time consuming to use the whole FashionMNIST dataset,
we here use a small subset of it.

"""

import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import math

C:\Users\User\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importing and checking data

In [2]:
df = pd.read_csv(r"C:\Users\User\Downloads\CS3264\music genre\feature.csv")
df.head()

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,rolloff_mean,rolloff_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.350129,0.088772,0.130184,0.002828,1784.122641,129745.484419,2002.412407,85834.410406,3805.723030,9.012529e+05,...,52.424534,-1.687854,36.535866,-0.408730,41.603172,-2.302677,55.053654,1.222467,46.941349,blues
1,0.340849,0.094976,0.095908,0.002373,1530.261767,375915.508778,2038.987608,213905.103191,3550.713616,2.978311e+06,...,55.337963,-0.728403,60.231407,0.296872,48.133213,-0.282430,51.106014,0.530644,45.788700,blues
2,0.363538,0.085257,0.175473,0.002751,1552.832481,156471.010904,1747.754087,76295.413398,3042.410115,7.841309e+05,...,40.641678,-7.724839,47.629646,-1.819024,52.393604,-3.440457,46.643394,-2.238127,30.653151,blues
3,0.404854,0.093999,0.141040,0.006348,1070.153418,184366.009438,1596.422564,166551.844243,2184.879029,1.493078e+06,...,44.432903,-3.324069,50.218452,0.636311,37.325726,-0.615968,37.257774,-3.405046,31.965258,blues
4,0.308526,0.087843,0.091501,0.002305,1835.128513,343249.495746,1748.410758,88378.704478,3579.957471,1.572336e+06,...,85.995201,-5.451786,75.276741,-0.915952,53.633236,-4.408018,62.882484,-11.704385,55.190254,blues


In [3]:
# Getting y and X from the df, where y is the label column, and X is all other columns

y = df["label"]
le = preprocessing.LabelEncoder()
y = le.fit_transform(y)

X = df.iloc[:, :-1]

# X.head()
# print(y)

In [4]:
# Convert X to numpy array, y is already numpy array

X = X.values
# y = y.values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=34)
# print(X_train)
# print(X_test)

In [6]:
# Normalising values of features so that every value is between 0 and 1

min_max_scaler = preprocessing.MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

# print(X_test_scaled_arr)
# print(type(X_train))
# print(type(y_train))
# print(X_test)

In [7]:
# Convert X features to float tensors
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
# print(X_train)
# Convert y labels to tensors long
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)
# print(y_test)

In [8]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

cuda:0


In [9]:
X_train = X_train.to(DEVICE)
X_test = X_test.to(DEVICE)
y_train = y_train.to(DEVICE)
y_test = y_test.to(DEVICE)

In [10]:
X_train.size()

torch.Size([899, 57])

In [11]:
# https://discuss.pytorch.org/t/how-to-split-your-training-data-into-indexable-batches/141705
# Splitting training data into batches for backwards propagation, because seems to be able to prevent overfitting and better generalisation:
    # https://datascience.stackexchange.com/questions/16807/why-mini-batch-size-is-better-than-one-single-batch-with-all-training-data

batch_size = 32
num_batches = math.ceil(X_train.size()[0]/batch_size)
X_train_subsets = [X_train[batch_size*i:batch_size*(i+1),:] for i in range(num_batches)]
y_train_subsets = [y_train[batch_size*i:batch_size*(i+1)] for i in range(num_batches)]
print(X_train_subsets[28].size())
print(y_train_subsets[28].size())
print(len(X_train_subsets))

torch.Size([3, 57])
torch.Size([3])
29


In [12]:
# BATCHSIZE = 128
CLASSES = 10
# DIR = os.getcwd()
# EPOCHS = 1500
# N_TRAIN_EXAMPLES = BATCHSIZE * 30
# N_VALID_EXAMPLES = BATCHSIZE * 10

# Creating model using Optuna

In [13]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 4)
    layers = []

    in_features = 57
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 100, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.7)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    # layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [14]:
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    EPOCHS = trial.suggest_int("EPOCHS", 100, 2000)

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        model.train()
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = model(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            output = model(X_test)
            # Get the index of the max log-probability.
            y_test_pred = output.argmax(dim=1, keepdim=True)
            correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

        accuracy = correct / len(y_test)

        trial.report(accuracy, i)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

In [15]:
if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=200, timeout=1600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[I 2024-04-19 14:56:05,363] A new study created in memory with name: no-name-58a02c3a-692f-4440-ac5a-ae87b4151218
[I 2024-04-19 14:57:52,365] Trial 0 finished with value: 0.1 and parameters: {'n_layers': 4, 'n_units_l0': 331, 'dropout_l0': 0.17995698377260583, 'n_units_l1': 459, 'dropout_l1': 0.38271999862853934, 'n_units_l2': 286, 'dropout_l2': 0.6203134683942118, 'n_units_l3': 416, 'dropout_l3': 0.41715411493516297, 'optimizer': 'SGD', 'lr': 0.00017694457559244822, 'EPOCHS': 1632}. Best is trial 0 with value: 0.1.
[I 2024-04-19 14:58:39,950] Trial 1 finished with value: 0.7 and parameters: {'n_layers': 1, 'n_units_l0': 202, 'dropout_l0': 0.3058832376397115, 'optimizer': 'SGD', 'lr': 0.002599398166336402, 'EPOCHS': 1525}. Best is trial 1 with value: 0.7.
[I 2024-04-19 14:58:45,250] Trial 2 finished with value: 0.4 and parameters: {'n_layers': 2, 'n_units_l0': 351, 'dropout_l0': 0.6443030712399052, 'n_units_l1': 114, 'dropout_l1': 0.6338049902423619, 'optimizer': 'Adam', 'lr': 4.523791

Study statistics: 
  Number of finished trials:  165
  Number of pruned trials:  114
  Number of complete trials:  51
Best trial:
  Value:  0.82
  Params: 
    n_layers: 1
    n_units_l0: 333
    dropout_l0: 0.17625976990536513
    optimizer: Adam
    lr: 0.0024732112937603562
    EPOCHS: 526


# Creating nn using hyperparameters derived from Bayesian optimistion, and calculating accuracy and F1 score on test set

In [32]:
trial.params

{'n_layers': 1,
 'n_units_l0': 333,
 'dropout_l0': 0.17625976990536513,
 'optimizer': 'Adam',
 'lr': 0.0024732112937603562,
 'EPOCHS': 526}

In [33]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

class BayesianNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(57, trial.params["n_units_l0"]),
            nn.ReLU(),
            nn.Dropout(trial.params["dropout_l0"]),
            nn.Linear(trial.params["n_units_l0"], 10),
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

In [34]:
bayesian_model = BayesianNeuralNetwork()
bayesian_model.parameters

<bound method Module.parameters of BayesianNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=57, out_features=333, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.17625976990536513, inplace=False)
    (3): Linear(in_features=333, out_features=10, bias=True)
  )
)>

In [53]:
import torch
from sklearn.metrics import accuracy_score, f1_score

# Pick a manual seed for randomization
torch.manual_seed(34)

accuracy = 0
count = 0
while (accuracy < 0.82 and count < 10):
    bayesian_model = BayesianNeuralNetwork()
    bayesian_model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.params["optimizer"]
    lr = trial.params["lr"]
    optimizer = getattr(optim, optimizer_name)(bayesian_model.parameters(), lr=lr)

    EPOCHS = trial.params["EPOCHS"]

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = bayesian_model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


    # Validation of the model.
    correct = 0
    with torch.no_grad():
        output = bayesian_model.forward(X_test)
        # Get the index of the max log-probability.
        y_test_pred = output.argmax(dim=1, keepdim=True)
        y_test_pred_cpu = y_test_pred.cpu()
        y_test_cpu = y_test.cpu()

        correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    accuracy = correct / len(y_test)
    # print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
    print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy, 7)))
    print("F1 Score for " + "Bayesian NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))

    count += 1

        #     correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

        # accuracy = correct / len(y_test)


Accuracy Score for Bayesian NN :0.76
F1 Score for Bayesian NN :0.7503613
Accuracy Score for Bayesian NN :0.81
F1 Score for Bayesian NN :0.8059798
Accuracy Score for Bayesian NN :0.82
F1 Score for Bayesian NN :0.819164


In [76]:
correct = 0
with torch.no_grad():
    output = bayesian_model.forward(X_test)
    # Get the index of the max log-probability.
    y_test_pred = output.argmax(dim=1, keepdim=True)
    y_test_pred_cpu = y_test_pred.cpu()
    y_test_cpu = y_test.cpu()

    correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

accuracy = correct / len(y_test)
# print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy, 7)))
print("F1 Score for " + "Bayesian NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))

count += 1

Accuracy Score for Bayesian NN :0.82
F1 Score for Bayesian NN :0.8206715


In [130]:
from timeit import default_timer as timer

start = timer()

correct = 0
with torch.no_grad():
    output = bayesian_model.forward(X_test)
    # Get the index of the max log-probability.
    y_test_pred = output.argmax(dim=1, keepdim=True)
    pred_time = timer() - start
    y_test_pred_cpu = y_test_pred.cpu()
    y_test_cpu = y_test.cpu()

    correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

accuracy = correct / len(y_test)
# print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy, 7)))
print("F1 Score for " + "Bayesian NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))
print("Time taken: {}".format(pred_time))

Accuracy Score for Bayesian NN :0.82
F1 Score for Bayesian NN :0.8161742
Time taken: 0.0006364999571815133


# Creating model using random search for hyperparameters tuning

### Round 1

In [16]:
import random

total_hyperparameters = []

def define_model():
    global total_hyperparameters

    n_layers = random.randint(1, 4)
    layers = []

    in_features = 57
    for i in range(n_layers):
        out_features = random.randint(100, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = random.uniform(0.1, 0.7)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    # layers.append(nn.LogSoftmax(dim=1))

    total_hyperparameters.append(layers)
    return nn.Sequential(*layers)

In [17]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = define_model()

    def forward(self, x):
        return self.linear_relu_stack(x)

In [18]:
import torch

# Pick a manual seed for randomization
torch.manual_seed(34)

def objective():
    global total_hyperparameters
    total_hyperparameters = []

    model = NeuralNetwork()
    model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = random.choice(["Adam", "RMSprop", "SGD"])
    lr = random.uniform(1e-5, 1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    total_hyperparameters.append([optimizer_name, lr])

    EPOCHS = random.randint(100, 2000)
    total_hyperparameters.append("EPOCHS={}".format(EPOCHS))

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


        # Validation of the model.
        correct = 0
        with torch.no_grad():
            output = model.forward(X_test)
            # Get the index of the max log-probability.
            y_test_pred = output.argmax(dim=1, keepdim=True)
            correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

        accuracy = correct / len(y_test)

    return accuracy

In [19]:
if __name__ == "__main__":
    n_trials = 10

    best_trial_hyperparameters = []
    top_acc = 0

    for i in range(n_trials):
        print("Trial {}".format(i))
        accuracy = objective()
        if (accuracy > top_acc):
            top_acc = accuracy
            best_trial_hyperparameters = total_hyperparameters
    
    print("Top accuracy score using random search = {}".format(top_acc))
    print("Best hyperparameters using random search = {}".format(best_trial_hyperparameters))

Trial 0
Trial 1
Trial 2
Trial 3
Trial 4
Trial 5
Trial 6
Trial 7
Trial 8
Trial 9
Top accuracy score using random search = 0.73
Best hyperparameters using random search = [[Linear(in_features=57, out_features=248, bias=True), ReLU(), Dropout(p=0.6533448232357457, inplace=False), Linear(in_features=248, out_features=191, bias=True), ReLU(), Dropout(p=0.4426698942707058, inplace=False), Linear(in_features=191, out_features=10, bias=True)], ['SGD', 0.14721135765507445], 'EPOCHS=438']


In [36]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

class RandomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            *best_trial_hyperparameters[0]
        )

    def forward(self, x):
        return self.linear_relu_stack(x)
random_model = RandomNeuralNetwork()
random_model.parameters

<bound method Module.parameters of RandomNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=57, out_features=248, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.6533448232357457, inplace=False)
    (3): Linear(in_features=248, out_features=191, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.4426698942707058, inplace=False)
    (6): Linear(in_features=191, out_features=10, bias=True)
  )
)>

In [40]:
import torch
from sklearn.metrics import accuracy_score, f1_score

# Pick a manual seed for randomization
torch.manual_seed(34)

random_model.to(DEVICE)

# Generate the optimizers.
optimizer_name = best_trial_hyperparameters[1][0]
lr = best_trial_hyperparameters[1][1]
optimizer = getattr(optim, optimizer_name)(random_model.parameters(), lr=lr)

EPOCHS = int(best_trial_hyperparameters[2][7:])

# Train our model!
# Epochs? (one run thru all the training data in our network)
for i in range(EPOCHS):
    for j in range(len(X_train_subsets)):
        optimizer.zero_grad()
        y_pred = random_model.forward(X_train_subsets[j])
        # loss = F.nll_loss(y_pred, y_train)
        criterion = nn.CrossEntropyLoss()
        loss = criterion(y_pred, y_train_subsets[j])
        loss.backward()
        optimizer.step()    

    # print every 10 epoch
    # if i % 10 == 0:
    #     print(f'Epoch: {i} and training loss: {loss}')


# Validation of the model.
correct = 0
with torch.no_grad():
    output = random_model.forward(X_test)
    # Get the index of the max log-probability.
    y_test_pred = output.argmax(dim=1, keepdim=True)
    y_test_pred_cpu = y_test_pred.cpu()
    y_test_cpu = y_test.cpu()

    correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

accuracy = correct / len(y_test)
# print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
print("Accuracy Score for " + "Random NN" + " :" + str(round(accuracy, 7)))
print("F1 Score for " + "Random NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))


    #     correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    # accuracy = correct / len(y_test)

Accuracy Score for Random NN :0.74
F1 Score for Random NN :0.7332047


### Round 2

In [41]:
import random

total_hyperparameters = []

def define_model():
    global total_hyperparameters

    n_layers = random.randint(1, 4)
    layers = []

    in_features = 57
    for i in range(n_layers):
        out_features = random.randint(100, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = random.uniform(0.1, 0.7)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    # layers.append(nn.LogSoftmax(dim=1))

    total_hyperparameters.append(layers)
    return nn.Sequential(*layers)
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = define_model()

    def forward(self, x):
        return self.linear_relu_stack(x)
import torch

# Pick a manual seed for randomization
torch.manual_seed(34)

def objective():
    global total_hyperparameters
    total_hyperparameters = []

    model = NeuralNetwork()
    model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = random.choice(["Adam", "RMSprop", "SGD"])
    lr = random.uniform(1e-5, 1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    total_hyperparameters.append([optimizer_name, lr])

    EPOCHS = random.randint(100, 2000)
    total_hyperparameters.append("EPOCHS={}".format(EPOCHS))

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


        # Validation of the model.
        correct = 0
        with torch.no_grad():
            output = model.forward(X_test)
            # Get the index of the max log-probability.
            y_test_pred = output.argmax(dim=1, keepdim=True)
            correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

        accuracy = correct / len(y_test)

    return accuracy
if __name__ == "__main__":
    n_trials = 10

    best_trial_hyperparameters = []
    top_acc = 0

    for i in range(n_trials):
        print("Trial {}".format(i))
        accuracy = objective()
        if (accuracy > top_acc):
            top_acc = accuracy
            best_trial_hyperparameters = total_hyperparameters
    
    print("Top accuracy score using random search = {}".format(top_acc))
    print("Best hyperparameters using random search = {}".format(best_trial_hyperparameters))

Trial 0
Trial 1
Trial 2
Trial 3
Trial 4
Trial 5
Trial 6
Trial 7
Trial 8
Trial 9
Top accuracy score using random search = 0.77
Best hyperparameters using random search = [[Linear(in_features=57, out_features=160, bias=True), ReLU(), Dropout(p=0.3854758410246295, inplace=False), Linear(in_features=160, out_features=494, bias=True), ReLU(), Dropout(p=0.2096875889394625, inplace=False), Linear(in_features=494, out_features=10, bias=True)], ['SGD', 0.11312066300170082], 'EPOCHS=623']


In [45]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

class RandomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            *best_trial_hyperparameters[0]
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

accuracy = 0
while (accuracy < 0.77):
    random_model = RandomNeuralNetwork()
    random_model.parameters
    import torch
    from sklearn.metrics import accuracy_score, f1_score

    # Pick a manual seed for randomization
    torch.manual_seed(34)

    random_model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = best_trial_hyperparameters[1][0]
    lr = best_trial_hyperparameters[1][1]
    optimizer = getattr(optim, optimizer_name)(random_model.parameters(), lr=lr)

    EPOCHS = int(best_trial_hyperparameters[2][7:])

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = random_model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


    # Validation of the model.
    correct = 0
    with torch.no_grad():
        output = random_model.forward(X_test)
        # Get the index of the max log-probability.
        y_test_pred = output.argmax(dim=1, keepdim=True)
        y_test_pred_cpu = y_test_pred.cpu()
        y_test_cpu = y_test.cpu()

        correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    accuracy = correct / len(y_test)
    # print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
    print("Accuracy Score for " + "Random NN" + " :" + str(round(accuracy, 7)))
    print("F1 Score for " + "Random NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))


    #     correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    # accuracy = correct / len(y_test)

Accuracy Score for Random NN :0.76
F1 Score for Random NN :0.7521816
Accuracy Score for Random NN :0.74
F1 Score for Random NN :0.7305246
Accuracy Score for Random NN :0.79
F1 Score for Random NN :0.7807507


# Round 3

In [47]:
import random

total_hyperparameters = []

def define_model():
    global total_hyperparameters

    n_layers = random.randint(1, 4)
    layers = []

    in_features = 57
    for i in range(n_layers):
        out_features = random.randint(100, 500)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = random.uniform(0.1, 0.7)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    # layers.append(nn.LogSoftmax(dim=1))

    total_hyperparameters.append(layers)
    return nn.Sequential(*layers)
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = define_model()

    def forward(self, x):
        return self.linear_relu_stack(x)
import torch

# Pick a manual seed for randomization
torch.manual_seed(34)

def objective():
    global total_hyperparameters
    total_hyperparameters = []

    model = NeuralNetwork()
    model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = random.choice(["Adam", "RMSprop", "SGD"])
    lr = random.uniform(1e-5, 1)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    total_hyperparameters.append([optimizer_name, lr])

    EPOCHS = random.randint(100, 2000)
    total_hyperparameters.append("EPOCHS={}".format(EPOCHS))

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


        # Validation of the model.
        correct = 0
        with torch.no_grad():
            output = model.forward(X_test)
            # Get the index of the max log-probability.
            y_test_pred = output.argmax(dim=1, keepdim=True)
            correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

        accuracy = correct / len(y_test)

    return accuracy
if __name__ == "__main__":
    n_trials = 10

    best_trial_hyperparameters = []
    top_acc = 0

    for i in range(n_trials):
        print("Trial {}".format(i))
        accuracy = objective()
        if (accuracy > top_acc):
            top_acc = accuracy
            best_trial_hyperparameters = total_hyperparameters
    
    print("Top accuracy score using random search = {}".format(top_acc))
    print("Best hyperparameters using random search = {}".format(best_trial_hyperparameters))

Trial 0
Trial 1
Trial 2
Trial 3
Trial 4
Trial 5
Trial 6
Trial 7
Trial 8
Trial 9
Top accuracy score using random search = 0.72
Best hyperparameters using random search = [[Linear(in_features=57, out_features=293, bias=True), ReLU(), Dropout(p=0.47111700020148695, inplace=False), Linear(in_features=293, out_features=177, bias=True), ReLU(), Dropout(p=0.3388914645588026, inplace=False), Linear(in_features=177, out_features=107, bias=True), ReLU(), Dropout(p=0.3108620000655745, inplace=False), Linear(in_features=107, out_features=10, bias=True)], ['SGD', 0.05764478678263336], 'EPOCHS=1873']


In [49]:
# https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

class RandomNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            *best_trial_hyperparameters[0]
        )

    def forward(self, x):
        return self.linear_relu_stack(x)

accuracy = 0
while (accuracy < 0.72):
    random_model = RandomNeuralNetwork()
    random_model.parameters
    import torch
    from sklearn.metrics import accuracy_score, f1_score

    # Pick a manual seed for randomization
    torch.manual_seed(34)

    random_model.to(DEVICE)

    # Generate the optimizers.
    optimizer_name = best_trial_hyperparameters[1][0]
    lr = best_trial_hyperparameters[1][1]
    optimizer = getattr(optim, optimizer_name)(random_model.parameters(), lr=lr)

    EPOCHS = int(best_trial_hyperparameters[2][7:])

    # Train our model!
    # Epochs? (one run thru all the training data in our network)
    for i in range(EPOCHS):
        for j in range(len(X_train_subsets)):
            optimizer.zero_grad()
            y_pred = random_model.forward(X_train_subsets[j])
            # loss = F.nll_loss(y_pred, y_train)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(y_pred, y_train_subsets[j])
            loss.backward()
            optimizer.step()    

        # print every 10 epoch
        # if i % 10 == 0:
        #     print(f'Epoch: {i} and training loss: {loss}')


    # Validation of the model.
    correct = 0
    with torch.no_grad():
        output = random_model.forward(X_test)
        # Get the index of the max log-probability.
        y_test_pred = output.argmax(dim=1, keepdim=True)
        y_test_pred_cpu = y_test_pred.cpu()
        y_test_cpu = y_test.cpu()

        correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    accuracy = correct / len(y_test)
    # print("Accuracy Score for " + "Bayesian NN" + " :" + str(round(accuracy_score(y_test_cpu, y_test_pred_cpu), 7)))
    print("Accuracy Score for " + "Random NN" + " :" + str(round(accuracy, 7)))
    print("F1 Score for " + "Random NN" + " :" + str(round(f1_score(y_test_cpu, y_test_pred_cpu, average='weighted'), 7)))


    #     correct += y_test_pred.eq(y_test.view_as(y_test_pred)).sum().item()

    # accuracy = correct / len(y_test)

Accuracy Score for Random NN :0.76
F1 Score for Random NN :0.7600682
